# Movie Review Classification

Problem Statement:           
Develop a classification model to predict whether the movie review was Positive or Negative labels by mining the text data.

<p><img src="resources/movie_review.jpg" style="width: 500px"></p>

Data Source:              
For this project we'll use the Cornell University Movie Review polarity dataset v2.0 obtained from http://www.cs.cornell.edu/people/pabo/movie-review-data/

## Perform imports and load the dataset
The dataset contains the text of 2000 movie reviews. 1000 are positive, 1000 are negative, and the text has been preprocessed as a tab-delimited file. Positive review is labeled as 'pos' and negative review is labeled as 'neg'

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics

In [2]:
m_reviews = pd.read_csv('datasets/moviereviews.tsv', sep='\t')
m_reviews.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [3]:
#The dataset contains 2000 observations and 2 columns
m_reviews.shape

(2000, 2)

### Example of a review

In [4]:
print(m_reviews['review'][0])

how do films like mouse hunt get into theatres ? 
isn't there a law or something ? 
this diabolical load of claptrap from steven speilberg's dreamworks studio is hollywood family fare at its deadly worst . 
mouse hunt takes the bare threads of a plot and tries to prop it up with overacting and flat-out stupid slapstick that makes comedies like jingle all the way look decent by comparison . 
writer adam rifkin and director gore verbinski are the names chiefly responsible for this swill . 
the plot , for what its worth , concerns two brothers ( nathan lane and an appalling lee evens ) who inherit a poorly run string factory and a seemingly worthless house from their eccentric father . 
deciding to check out the long-abandoned house , they soon learn that it's worth a fortune and set about selling it in auction to the highest bidder . 
but battling them at every turn is a very smart mouse , happy with his run-down little abode and wanting it to stay that way . 
the story alternates betwee

## Check for missing values:
Reviews could be missing if a reviewer declined to provide a comment with their review. Missing review can be either NaN or empty string

### Detect & remove NaN values:

In [5]:
# Check for the existence of NaN values
m_reviews.isnull().sum()

label      0
review    35
dtype: int64

35 records show **NaN**

In [6]:
m_reviews.dropna(inplace=True)
len(m_reviews)

1965

### Detect & remove empty strings

In [7]:
blanks = []
for i,lb,rv in m_reviews.itertuples(): 
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

m_reviews.drop(blanks, inplace=True)
print(str(len(blanks)) + ' blanks were removed')

27 blanks were removed


In [8]:
m_reviews['label'].value_counts()

neg    969
pos    969
Name: label, dtype: int64

## Split the data into train & test sets:

In [9]:
X = m_reviews['review']
y = m_reviews['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### List of general stop words

In [10]:
stopwords = ['a', 'about', 'an', 'and', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', \
             'even', 'ever', 'for', 'from', 'get', 'had', 'has', 'have', 'he', 'her', 'hers', 'his', \
             'how', 'i', 'if', 'in', 'into', 'is', 'it', 'its', 'just', 'me', 'my', 'of', 'on', 'or', \
             'see', 'seen', 'she', 'so', 'than', 'that', 'the', 'their', 'there', 'they', 'this', \
             'to', 'was', 'we', 'were', 'what', 'when', 'which', 'who', 'will', 'with', 'you']

## Build pipelines to vectorize the data, then train and fit a model

### Multinomial NB

We will start with by building a pipeline for Multinomial Naive Baiyes model. 

* The first step of the pipeline would to convert raw documents in to a matrix of TF-IDF features.
* Instatiate an instance of Multinomial Naive Baiyes model

In [11]:
# Naive Bayes:
text_clf_nb = Pipeline([('tfim_reviews', TfidfVectorizer(stop_words=stopwords)),
                     ('clf', MultinomialNB()),
])

text_clf_nb.fit(X_train, y_train)

# Form a prediction set
predictions = text_clf_nb.predict(X_test)

# Report the confusion matrix
print(metrics.confusion_matrix(y_test,predictions))

[[282  26]
 [105 227]]


In [12]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         neg       0.73      0.92      0.81       308
         pos       0.90      0.68      0.78       332

    accuracy                           0.80       640
   macro avg       0.81      0.80      0.79       640
weighted avg       0.82      0.80      0.79       640



In [13]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.7953125


Naive Bayes gave a  better-than-average result at 79.5% for classifying reviews as positive or negative based on text alone. 

Now try to improve the performance using another model ie. LinearSVC

### Linear SVC

We will start with by building a pipeline for Linear SVC. 

* The first step of the pipeline would to convert raw documents in to a matrix of TF-IDF features.
* Instatiate an instance of Linear Support Vector Classifier model

In [14]:
# Linear SVC:
text_clf_lsvc = Pipeline([('tfim_reviews', TfidfVectorizer(stop_words=stopwords)),
                     ('clf', LinearSVC()),
])

text_clf_lsvc.fit(X_train, y_train)

# Form a prediction set
predictions = text_clf_lsvc.predict(X_test)

# Report the confusion matrix
print(metrics.confusion_matrix(y_test,predictions))

[[256  52]
 [ 48 284]]


In [15]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         neg       0.84      0.83      0.84       308
         pos       0.85      0.86      0.85       332

    accuracy                           0.84       640
   macro avg       0.84      0.84      0.84       640
weighted avg       0.84      0.84      0.84       640



In [16]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.84375


## Conclusion

Linear SVC correctly classified the reviews as positive or negative **84.3%** of the time which is an improvement over Multinomial Naive Baiyes

## Feed new data into a trained model for classification model
Since we have developed a fairly accurate model, it's time to feed new data through it. We'll write our own review, and see how accurately our model assigns a "positive" or "negative" label to it.

In [17]:
#Writing a negative review

myreview = "A movie I really wanted to love was terrible. \
I'm sure the producers had the best intentions, but the execution was lacking."

In [18]:
print(text_clf_lsvc.predict([myreview]))

['neg']


Thus, the model correctly classified the movie review as negative.